# Bloomberg BQuant Spotlight Webinar Series:  The Global Equity Rally - A China Case Study On Market Breadth 
This is a companion notebook to the [Global Equity Rally - A China Case Study On Market Breadth](https://blinks.bloomberg.com/screens/PLYR%20VOD%20336538940) webinar.

### Part 0: Import libraries

In [ ]:
import bql

import numpy as np
import pandas as pd

from IPython.display import display
import bqviz as bqv
from bqplot import (
    DateScale, LinearScale, Lines, Axis, Figure, 
)

In [ ]:
bq = bql.Service()

### Part 1: Setup Market Breadth Study

In [ ]:
# visualize the index level

start='2018-12-03'
end='2019-03-29'
universe = 'szcomp index'


request = bql.Request(universe,{universe:bq.data.px_last(start,end,fill="prev")})
response = bq.execute(request)
index_df = response[0].df()
index_df.set_index(['DATE'],inplace=True)
index_df.drop(['CURRENCY'],axis=1,inplace=True)
bqv.LinePlot(index_df).set_style().show()

In [ ]:
start='2018-12-03'
end='2019-03-29'
universe = 'szcomp index'


securities = bq.univ.members(universe)
date_range = {'dates' : bq.func.range(start,end)}   

# define market breadth as stocks hitting 20 days high and low
days = 20

px_last = bq.data.px_last(**date_range)
max_min = bq.data.maxmin(px_last,px_last,period=days)
max_ratio = max_min['maxline']/px_last
min_ratio = max_min['minline']/px_last      
hitting_high = (bq.func.if_(max_ratio==1,1,0)).group([px_last['date']]).sum()
hitting_low = (bq.func.if_(min_ratio==1,-1,0)).group([px_last['date']]).sum()


request = bql.Request(securities,{'hitting_high':hitting_high,'hitting_low':hitting_low})
response = bq.execute(request)

In [ ]:
len(response)

In [ ]:
df = bql.combined_df(response)
df.head()

In [ ]:
df.index = df.iloc[:,1]
daily_breadth_df = df.iloc[:,-2:]
bqv.BarPlot(daily_breadth_df,bar_type='stacked',title='Dailly No.of Stocks Hitting Highs and Lows').set_style().show()

### Part 2: Setup Model Portfolio

In [ ]:
def factor_lib(date_range,style):

#define style factors
#params should include dates=range() for most of factors to work
#output a list of data items based on requsted factor

    params = dict({'fill':'prev'},**date_range)
    
    params_cny = dict({'currency':'CNY'},**params)
    
    
    # define style/factors
    factor_dict = {      
        
        'growth':[bq.data.sales_growth(**params), 
                  bq.data.eps_growth(**params) 
                 ],
        
        'quality':[bq.data.operating_roic(**params),
                   bq.data.return_com_eqy(**params)
                  ],
        
        'size':[-bq.data.cur_mkt_cap(**params_cny),
                -bq.data.sales_rev_turn(**params_cny)
                ]                        
    }
    return factor_dict[style]

    
def factor_zscore(factor_list):
    
#convrt factor to zscore for scoring/cutting

    zscore_list = [bq.func.groupzscore(bq.func.first(factor.dropna())) for factor in factor_list]
    return sum(zscore_list)

In [ ]:
# output bql.request for quantitle 1 stocks based on size factor

start='2018-12-03'
end='2019-03-31'
universe = 'szcomp index'
style = 'growth'

securities = bq.univ.members(universe)
date_range = {'dates' : bq.func.range(start,end)}   
zscore = factor_zscore(factor_lib(date_range,style))
bins = bq.func.cut(zscore.group(),5).ungroup()

filtered = bq.univ.filter(securities,bins==1)
output = bq.data.name()

request = bql.Request(filtered,{'Output':output})
response = bq.execute(request)

In [ ]:
df = response[0].df()
len(df)

In [ ]:
df.head()

### Part 3: Work on 3 factors

In [ ]:
def market_breadth_request(universe,start,end,style):
    
# output bql.request for market breadth based on universe, start/end date, factor style and market breadth method
    
    securities = bq.univ.members(universe,dates=start)
    
    date_range = {'dates' : bq.func.range(start,end)}   
    zscore = factor_zscore(factor_lib(date_range,style))
    bins = bq.func.cut(zscore.group(),5).ungroup()
    filtered = bq.univ.filter(securities,bins==1)
    
    days = 20

    px_last = bq.data.px_last(**date_range)
    max_min = bq.data.maxmin(px_last,px_last,period=days)
    max_ratio = max_min['maxline']/px_last
    min_ratio = max_min['minline']/px_last      
    hitting_high = (bq.func.if_(max_ratio==1,1,0)).group([px_last['date']]).sum()
    hitting_low = (bq.func.if_(min_ratio==1,-1,0)).group([px_last['date']]).sum()
    net_high = hitting_high + hitting_low
       
    return bql.Request(filtered,{'Output':net_high})

In [ ]:
style_list = ['growth','quality','size']
response_dict = {}
start = '2018-12-03'
end = '2019-03-29'
universe = 'szcomp index'
for style in style_list:
    request = market_breadth_request(universe,start,end,style)
    print('request market breadth for {}...'.format(style))
    response_dict[style] = bq.execute(request)
print('all requests returned')

In [ ]:
df_list = []

for style,response in response_dict.items():
    df = response[0].df()
    df.rename(columns={'Output':style},inplace=True)
    df = pd.DataFrame(df[style])
    df_list.append(df)
    
display_df = pd.concat(df_list,axis=1)
display_df.index = pd.to_datetime(display_df.index,format='%Y-%m-%dT%H-%M-%SZ')

In [ ]:
display_df.head()

### part 4: Visualization

In [ ]:
x_date = DateScale()
y_ls = LinearScale()
y2_ls = LinearScale()



styles = Lines(x=display_df.index, y=np.array(display_df.cumsum()).T, 
               scales={'x': x_date, 'y':y_ls}, 
               line_style='solid',display_legend=True,labels=list(display_df.columns))

index_lvl = Lines(x=index_df.index, y=index_df, 
             scales={'x': x_date, 'y': y2_ls},
             stroke_width=3, colors=['white'], line_style='dashed',
             display_legend=True, labels=list(index_df.columns))

ax_x = Axis(scale=x_date, label='Date',visible=True)

# primary y axis
ax_y = Axis(scale=y_ls, orientation='vertical',
            grid_lines='none', label='cumulative No. Highs - No. Lows',tick_style={'font-size': 9})

# secondary y axis
ax_y2 = Axis(scale=y2_ls, orientation='vertical', tick_format='0.1f',
             label='Index Level',side='right',tick_style={'font-size': 9})

fig = Figure(marks=[styles, index_lvl], axes=[ax_x, ax_y,ax_y2], title='Market Breadth: {}'.format(universe),
       legend_location='top-left', layout={'width':'1000px','height':'600px'})


display(fig)